In [1]:
# Development - Transport Demand Module (TDM)

import pandas as pd

In [39]:
# import data - return TDSK
def return_country_data(country='Kenya', start_year=1990, end_year=2021):
    try:
        df = pd.read_excel("https://zenodo.org/records/10406893/files/TSDK_ALL.xlsx?download=1", sheet_name="Data")
        country_data = df[df['Country name'] == country][['Country name', 'Variable', 'Data code', 'Unit'] + [y for y in range(
            start_year, end_year)]]
        return country_data
    except Exception as e:
        return f"an error occurred (probably the source data at URL has changed): {e}"

In [40]:
# these functions return vectors of particular variables - gdp, population, etc. - the ones that are *useful* from TSDK.

# return gdp per year (total) for country, for which data exists.
def return_gdp(df):
    gdp = pd.Series(df[df['Data code'] == 'GDP_TOT'].iloc[0])
    return gdp.dropna()


# return total population and share urb/rur
def return_pop(df):
    pop = df[df['Data code'].isin(['POP_TOT', 'POP_URB', 'POP_RUR'])]
    return pop.dropna()


# return pkm_road (total) for country, for which data exists. Amalgamate sources.
def return_pkm_road(df):
    pkm_road = (df[df['Data code'] == 'ROAD_PA_MOV'].set_index(['Country name', 'Data code']).stack().groupby
                (level=[1, 2]).first().unstack())
    return pkm_road


# return pkm_rail (total) for country, for which data exists. Amalgamate sources.
def return_pkm_rail(df):
    pkm_road = (df[df['Data code'] == 'ROAD_PA_MOV'].set_index(['Country name', 'Data code']).stack().groupby
                (level=[1, 2]).first().unstack())
    return pkm_road

In [41]:
# logical test as to whether there is mode split data in the TSDK
def mode_share_test(df):
    if  all(v in df['Data code'].tolist() for v in ['ROAD_PA_MOTORC', 'ROAD_PA_CAR', 'ROAD_PA_BUS']):
        return True
    else:
        return False

In [42]:
# dictionary to hold values of mode shares by distance (pkm based) by country and mode
mode_share_pkm = {'Kenya': {'BUS': 0.55, 'CAR': 0.2, 'MOTO': 0.1, 'WALK': 0.08, 'BIKE': 0.02, 'AIR': 0.03, 'SHIP': 0.02}}

In [37]:
# TODO: run mode_share_test --> if true, then use data in TSDK. If false, use mode_share_km

country = 'Kenya'

df = return_country_data(country=country)

if mode_share_test(df):
    # do something
    print('dream on son')
else:
    for mode in mode_share_pkm[country]:
        mode_share_pkm[country][mode] * return_pkm_road(df)
        
        

True